## 증권 종목별 데이터 한번에 다 모으기. 
- author : 편도훈

In [44]:
from selenium import webdriver
import selenium
import pandas as pd
import numpy as np
import datetime
import time
from tqdm import tqdm

## 주소별로 원하는 정보 찾기.
- 종목코드 : code
- 시가총액 : market_cap(replace로 ,빼기)
- 상장일 : reg_day
- PER : per(소수점 두번째 자리까지)
- kospi 여부

In [36]:
##### 업종별 이름, 사이트 다 모으기.

# 크롬드라이버가 연결되어 있는 주소 저장
chromedriver ='E:/big12/python-project/note/driver/chromedriver.exe'

# 크롬드라이버 연결
driver = webdriver.Chrome(chromedriver)

#업종 사이트 연결
driver.get('https://finance.naver.com/sise/sise_group.nhn?type=upjong')

#업종의 tbody에는 업종별로 사이트를 저장하는 a태그들이 있다. 그것을 꺼낸다.
a_tags = driver.find_element_by_tag_name('tbody').find_elements_by_tag_name('a')

category_list =[]

#업종의 주소와 이름을 저장
for a in a_tags:
    category_list.append([a.get_attribute('href'),a.text])

#구글 드라이버를 꼭 닫아주기.
driver.close()

In [42]:
# 여기서부터 다시하기

# function 생성.
def add_category_info(name_website):
    #파라미터로 들어온 것은 [웹사이트,이름]으로 구성되어 있는 리스트 이다.
    #크롬드라이버 주소 저장
    chromedriver ='E:/big12/python-project/note/driver/chromedriver.exe'
    #크롬드라이버 object 생성
    driver = webdriver.Chrome(chromedriver)
    
    #웹사이트로 가기.
    driver.get(name_website[0])
    #특정업종사이트의 name_area의 a태그에는 종목의 주소와 이름이 저장되어있다.
    tmp = driver.find_elements_by_class_name('name_area')
    #종목 주소 저장
    address = [x.find_element_by_tag_name('a').get_attribute('href') for x in tmp]
    #종목 이름 저장
    name = [x.find_element_by_tag_name('a').text for x in tmp]
    
    #변수를 저장할 빈리스트
    code_list =[]
    market_cap_list =[]
    reg_day_list=[]
    per_list=[]
    kospi_list=[]
    name_list=[]
    
    #주소를 for문으로 돌린다.
    for addr in address:
        
        #드라이버로 1종목의 주소에 들어간다.
        driver.get(addr)
        #잠시대기
        time.sleep(0.5)
        
        
        #code라는 곳에는 업종코드가 저장
        code_list.append(driver.find_element_by_class_name('code').text)
        
        #wrap_companmy에 회사이름이 저장 되어 있따
        name_list.append(driver.find_element_by_class_name('wrap_company').find_element_by_tag_name('a').text)
        
        #_market_sum에는 시가총액이 들어가 있다.
        market_cap = driver.find_element_by_id('_market_sum').text
        
        #2가지 예외처리를 한다. 4조 9,222같은 경우에는 띄어쓰기를 기준으로 두개를 나누어주고 콤마를 제외하고 붙여야한다.
        if(market_cap.find('조')!=-1):
            tmp_list = market_cap.split(' ')
            market_cap = tmp_list[0][:-1]+tmp_list[1][:]
        if(market_cap.find(',')!=-1):
            market_cap = market_cap.replace(",","")
        
        #리스트에 저장해준다.    
        market_cap_list.append(int(market_cap))
        
        #per은 N/A인 경우가 있다. N/A인 경우는 해당 id 자체가 없기 때문에 try를 사용하여 except가 발생한다면 -1.0으로 대체한다.
        try:

            per = float(driver.find_element_by_id('_per').text)
        except:
            #na는 -1로 치환한다.
            per =-1.0
        finally:
            #어쨌든 저장
            per_list.append(per)
            
        #kospi라는 클래스가 있거나 kosdaq이라는 클래스가 있다. 어차피 둘 중 하나만 있기 때문에 
        #코스피 클래스가 없으면 코스닥으로 넘어간다.
        try:
            driver.find_element_by_class_name('kospi')
            kospi_list.append('kospi')
            
        except:
            kospi_list.append('kosdaq')

    #데이터프레임에 저장한다.
    df = pd.DataFrame(
        {
            'code' : code_list,
            'name' :name_list,
            'market_cap' : market_cap_list,
            'per' : per_list,
            'class' : kospi_list
        }
    )

    #상장일은 ajax처리가 되어 다른 사이트에 있다. 그 자료를 따로 받아주도록 하자.
    for code in code_list:
        #code_list에 저장되어 있는 것을 활용하여
        code= code
        
        #이 사이트에서 code만 바꾸어준다.
        reg_site = 'https://navercomp.wisereport.co.kr/v2/company/c1020001.aspx?cmp_cd={code}&cn='.format(code = code)
        
        #드라이버를 사용하여 해당사이트로 이동
        driver.get(reg_site)
        #상장일의 클래스는 클래스를 2개를 사용한다. c2 txt
        tmp_reg = driver.find_elements_by_class_name('c2.txt')[1].text
        
        #상장일이 좀 복잡하게 되어있던데 까먹었다. 이렇게 하면 되더라고요...
        reg_day_list.append(tmp_reg.split(' ')[2][:-1])
    
    
    #포문을 벗어나고 상장일을 df에 등록해준다.
    df['reg_day']=reg_day_list
    #코드에도 A를 추가해준다.(엑셀에서 숫자로 인식하여 0을 빼벌임)
    df['code'] = [ 'A'+x for x in df['code']]
    
    #업종의 이름으로 저장
    df.to_csv('./categories/'+name_website[1]+'.csv', encoding='utf-8')
    
    #꼭 닫아주기로 하자.
    driver.close()
    
    ###close와 open을 어디다 넣어줘야하는지 고민

In [38]:
category_list.pop(20)

print(len(category_list))

category_list[:5]


78


[['https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=109',
  '창업투자'],
 ['https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=44',
  '자동차'],
 ['https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=166',
  '출판'],
 ['https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=222',
  '디스플레이패널'],
 ['https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=36',
  '자동차부품']]

In [43]:
###category_list에 인덱싱 지정하셔도 됩니당


for name_website in tqdm(category_list):
    add_category_info(name_website)

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [46:38<00:00, 35.88s/it]


In [ ]:
pd.read_csv('./categories/', encoding='utf-8')

## 기타는 상장일이 없으니 빼주세용

In [2]:
# from multiprocessing import Pool
# 아 왜 안대냐귱
# cat1=[]
# cat2=[]
# cat3=[]
# cat4=[]

# for i in range(len(category_list)):
#     if i%4==0:
#         cat1.append(category_list[i])
#     elif i%4==1:
#         cat2.append(category_list[i])
#     elif i%4==2:
#         cat3.append(category_list[i])
#     elif i%4==3:
#         cat4.append(category_list[i])    
# print(len(cat1),cat1[1])

# sorted_categories = [cat1,cat2,cat3,cat4]
# sorted_categories

# pool = Pool(processes=4)

# pool.map(add_category_info,sorted_categories)